In [1]:
# Import dependencies
import os
from src.utils.gpu_config import configure_gpu, optimize_tensorflow

configure_gpu()
optimize_tensorflow()

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2025-04-22 19:46:29.747397: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-22 19:46:30.062630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745343990.164635    3412 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745343990.193751    3412 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745343990.433793    3412 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 


GPU configuration successful!
Device: /physical_device:GPU:0
Memory limit: 18.0 GB

TensorFlow optimizations status:
- CPU Threads: 19 intra-op, 12 inter-op (32 cores total)
- XLA JIT: enabled (autoclustering)
- Synchronous Execution: enabled

GPU Device Details:
- Device: GPU /physical_device:GPU:0
- Memory limit: 18.0 GB


I0000 00:00:1745343993.937901    3412 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19327352832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE, TF_MAX_EPOCHS
from src.model.tensorflow_utils import load_dataset, optimize_dataset, count_dataset_size, get_num_classes, get_metrics
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_early_stopping
from src.model.tensorflow_utils import start_or_resume_training, apply_augmentation_to_dataset
from src.model.densnet.tensorflow_dense_net_121 import build_densenet121

# Input Data
initial_epoch   = 5
resume_training = True
checkpoint_path = '/home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV1_1/checkpoints/cp-0004.keras'
set_name        = "DenseNetV1"
model_name      = f"{set_name}_1"
num_classes     = get_num_classes()

train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

I0000 00:00:1745343993.976576    3412 cuda_executor.cc:479] failed to allocate 18.00PiB (20266198323167232 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745343993.976604    3412 cuda_executor.cc:479] failed to allocate 16.20PiB (18239577202360320 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745343993.976608    3412 cuda_executor.cc:479] failed to allocate 14.58PiB (16415619482124288 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745343993.976610    3412 cuda_executor.cc:479] failed to allocate 13.12PiB (14774057211789312 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745343993.976612    3412 cuda_executor.cc:479] failed to allocate 11.81PiB (13296650738991104 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745343993.976614    3412 cuda_executor.cc:

In [3]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Disable Advanced Augmentations
probability = {
    "intensity_scaling": 0,
    "adaptive_histogram": 0,
    "zoom": 0,
    "cutout": 0
}

train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds, probability)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds   = optimize_dataset(val_ds, TF_BATCH_SIZE)

2025-04-22 19:46:36.232962: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 1048576
2025-04-22 19:47:32.750254: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
2025-04-22 19:47:38.782532: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Counting samples: 71842 samples [01:02, 1147.59 samples/s]
2025-04-22 19:47:52.244443: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Counting samples: 15431 samples [00:13, 1146.37 samples/s]


In [4]:
# Setup Model Deps
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
metrics           = get_metrics()

# Setup compile arguments
loss           = BinaryCrossentropy(from_logits=False)
reduce_lr      = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)
early_stopping = setup_early_stopping()

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

2025-04-22 19:47:52 - INFO - Resuming from existing metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV1_1/train_metrics.csv
2025-04-22 19:47:52 - INFO - Resuming from existing validation metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV1_1/val_metrics.csv
2025-04-22 19:47:52 - INFO - Cleaned training metrics file, kept 2244 records before epoch 5
2025-04-22 19:47:52 - INFO - Cleaned validation metrics file, kept 4 records before epoch 5
2025-04-22 19:47:52 - INFO - Found 2244 existing training records
2025-04-22 19:47:52 - INFO - Found 4 existing validation records


In [5]:
# Model Training
model          = build_densenet121(num_classes)
optimizer      = Adam(learning_rate=1e-4)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    TF_MAX_EPOCHS,
    steps_per_epoch, 
    validation_steps, 
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, early_stopping], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-04-22 19:47:53 - INFO - Resuming from existing metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV1_1/train_metrics.csv
2025-04-22 19:47:53 - INFO - Resuming from existing validation metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV1_1/val_metrics.csv
2025-04-22 19:47:54 - INFO - Cleaned training metrics file, kept 2244 records before epoch 5
2025-04-22 19:47:54 - INFO - Cleaned validation metrics file, kept 4 records before epoch 5
2025-04-22 19:47:54 - INFO - Found 2244 existing training records
2025-04-22 19:47:54 - INFO - Found 4 existing validation records


Loading full model from checkpoint: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV1_1/checkpoints/cp-0004.keras


2025-04-22 19:47:55 - INFO - 
=== Training Started ===

2025-04-22 19:47:55 - INFO - Batch Size: 128
2025-04-22 19:47:55 - INFO - Optimizer: Adam
2025-04-22 19:47:55 - INFO - 

2025-04-22 19:47:55 - INFO - 
=== Starting Epoch 5 ===



Epoch 5/100


I0000 00:00:1745344083.649426    3503 service.cc:152] XLA service 0x7790580021e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745344083.649440    3503 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
I0000 00:00:1745344083.666985    3503 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745344083.745390    3503 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-04-22 19:48:22.972407: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:10: Filling up shuffle buffer (this may take a while): 20860 of 25000
2025-04-22 19:48:24.853449: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
2025-04-22 19:48:26.113136: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to 

...

2025-04-22 21:47:48.785578: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 77078016 bytes after encountering the first element of size 77078016 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2025-04-22 21:47:54.126055: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:10: Filling up shuffle buffer (this may take a while): 8081 of 25000
2025-04-22 21:48:04.127731: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:10: Filling up shuffle buffer (this may take a while): 13800 of 25000
2025-04-22 21:48:16 - INFO - 
=== Epoch 47 Summary ===
2025-04-22 21:48:16 - INFO - Time: 162.82s
2025-04-22 21:48:16 - INFO - Training   - accuracy: 0.9610 - auc: 0.9364 - f1_score: 0.7163 - loss: 0.1414 - precision: 0.8547 - recall: 0.6562 - learning_rate: 0.0000
2025-04-22 21:48:16 - INFO - Validation - accur

561/561 ━━━━━━━━━━━━━━━━━━━━ 163s 290ms/step - accuracy: 0.9611 - auc: 0.9361 - f1_score: 0.7167 - loss: 0.1413 - precision: 0.8531 - recall: 0.6574 - val_accuracy: 0.9202 - val_auc: 0.7204 - val_f1_score: 0.4174 - val_loss: 0.3185 - val_precision: 0.5427 - val_recall: 0.3977 - learning_rate: 1.0000e-06


2025-04-22 21:48:16 - INFO - 
=== Training Completed! ===

2025-04-22 21:48:16 - INFO - Final Metrics: accuracy: 0.9610 - auc: 0.9364 - f1_score: 0.7163 - loss: 0.1414 - precision: 0.8547 - recall: 0.6562 - val_accuracy: 0.9202 - val_auc: 0.7204 - val_f1_score: 0.4174 - val_loss: 0.3185 - val_precision: 0.5427 - val_recall: 0.3977



Epoch 47: early stopping
Restoring model weights from the end of the best epoch: 32.
Epoch 47: early stopping
Restoring model weights from the end of the best epoch: 32.


2025-04-22 21:48:20.257078: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
2025-04-22 21:48:20.388614: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 77078016 bytes after encountering the first element of size 77078016 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
